# Choice simulation work

Sam Maurer, October 2018

This notebook contains benchmarks, feature development, and testing for ChoiceModels PR #TK, related to issue #TK.

In [3]:
import numpy as np
import pandas as pd

In [5]:
import choicemodels
print(choicemodels.__version__)

0.2.dev3


### Benchmark df.apply vs matrix math for chooser-level random draws

There's no `numpy` function to perform simultaneous random draws from K distinct probability distributions, which we often need to do to simulate choices for K choosers.

Fletcher wrote an implementation using matrix math for `urbansim.urbanchoice.mnl`, which I refactored and generalized in `choicemodels.tools`. 

But I realized that in other places, like `urbansim.models.dcm`, we use `df.apply` for similar operations. This seems cleaner and more easily maintainable, and i'm curious how the performance compares. Maybe the matrix math implementation is only needed for things like GPU acceleration?

In [1]:
from choicemodels.tools import simulate_choices

In [51]:
def generate_probs(n_obs, n_alts):
    n_obs = int(n_obs)
    n_alts = int(n_alts)
    
    d = {'oid': np.repeat(np.arange(n_obs), n_alts),
         'aid': np.tile(np.arange(n_alts), n_obs),
         'probs': np.random.random(n_obs * n_alts)}

    df = pd.DataFrame(d)
    df['probs'] = df.probs.div(df.groupby('oid').probs.transform('sum'))
    return df.set_index(['oid','aid']).probs

print(generate_probs(2,3))

oid  aid
0    0      0.026966
     1      0.083292
     2      0.889742
1    0      0.052409
     1      0.436480
     2      0.511111
Name: probs, dtype: float64


In [41]:
probs = generate_probs(1e4, 100)

#### 1. Matrix implementation from choicemodels

In [42]:
%%time
c = simulate_choices(probs)
print(len(c))

10000
CPU times: user 22.3 ms, sys: 5.12 ms, total: 27.4 ms
Wall time: 25.7 ms


#### 2. df.apply

In [43]:
df = pd.DataFrame(probs).reset_index()

In [44]:
%%time
c = df.groupby('oid').apply(lambda x: np.random.choice(x.aid, p=x.probs))
print(len(c))

10000
CPU times: user 1.02 s, sys: 9.52 ms, total: 1.03 s
Wall time: 1.03 s


#### 3. Try keeping indexes to make it faster

In [45]:
def mkchoice(probs):
    return np.random.choice(probs.index.values, p=probs)

In [46]:
%%time
c = probs.groupby(level='oid', sort=False).apply(mkchoice)
print(len(c))

10000
CPU times: user 3.3 s, sys: 28.1 ms, total: 3.33 s
Wall time: 3.31 s


**df.apply is way slower!! At least 50x. We should use the matrix math implementation everywhere**